In [1]:
!pip install --upgrade --quiet pymongo
% matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import modelling.fetch as fetch

In [3]:
df = fetch.create_df(fetch.get_labelled_articles("209.177.92.45:80"))

In [1]:
import pandas as pd
from modelling.clustering import dbscan, vectorize
from modelling.utils import get_articles
from sklearn.cluster import DBSCAN
from sklearn.neighbors import radius_neighbors_graph
from pymongo import MongoClient
from datetime import datetime

In [2]:
src = 'tw'
date = datetime(2017, 10, 10)

In [3]:
collection = MongoClient('209.177.92.45:80')['newsfilter'].news
tweets = get_articles(collection, src=src, date_start = date)
bodies = list(map(lambda x: x['content'].get('body'), tweets))
vector = vectorize(bodies)

In [4]:
vector

<25210x44920 sparse matrix of type '<class 'numpy.int64'>'
	with 535343 stored elements in Compressed Sparse Row format>

In [5]:
mini = vector[:2000,:]
small = vector[:10000,:]

In [ ]:
%%time
ng = radius_neighbors_graph(small, 2.0, 'distance')

In [ ]:
%%time
ng = radius_neighbors_graph(small, 0.5, 'connectivity')

In [ ]:
%%time
ng = radius_neighbors_graph(mini, 5.0, 'distance')

In [ ]:
%%time
ng = radius_neighbors_graph(mini, 0.5, 'distance')

In [17]:
db = DBSCAN(eps = .5, min_samples = 2, metric = 'precomputed')

In [ ]:
fit = db.fit_predict(ng)

In [ ]:
zipped = zip(fit, bodies)
z = sorted(zipped, key = lambda a: a[0])
z.reverse()
pd.DataFrame(z, columns = ['cluster', 'body'])